# Toronto Business Analysis of Tea shop

## A. Introduction

### 1) Description of the problem and a discussion of the background. 

Toronto, the most populous city in Canada, with a population of 2,731,571 as of 2016. Current to 2016, the Toronto census metropolitan area (CMA), of which the majority is within the Greater Toronto Area (GTA), held a population of 5,928,040, making it Canada's most populous CMA.The Toronto metropolitan area was the second-fastest-growing metropolitan area in North America, adding 125,298 persons.

The Toronto region’s Food and Beverage sector employs more than 64,000 workers with annual wages totaling $3.2B. Sector businesses located within the city of Toronto account for more than 50 per cent of this workforce.

The amount of tea that Canadians drink has doubled in the past few decades. As of 2015, Canadians drink 85 litres per person per year, up from 79.4 litres in 2008, and only 36 litres in 1991! Tea surpassed both soft drinks and bottled water, and is only slightly less than the amount of coffee that Canadians drink. However it’s also interesting to note that the average amount of coffee drank by Canadians decreased from the previous year, while tea rose by several litres. 

There are same neighborhood some places have a high demand while there are others hardly getting any profit. So if I wanna open a tea store, where would be a nice place to open my first store? The objective of the following analysis is to check where would be popular in a determined neighborhood, and which neighborhood has a better acceptance for that kind. 

### 2)  Who would be interesting at the capstone?

This capstone would be interesting for those who wanna open a tea store and have no idea where to open.

  * Investor
  * Entrepreneur
  * Tea supplier
  * Asian restaurants

### 3) Data that you can get from this capstone 

  * Top venues of Toronto in different sectors
  * Market demand (cluster)
  * Best place to open a tea store
  * Where do you have more competitor
  * etc.

### 4) Description of the data and how it will be used to solve the problem.

Considering the problem mentioned, the following data will be used to solve the problem:

1. From Wikipedia, I have found a complete list of postal codes of Toronto, followed by theirs borough and neighborhood name: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
2. I used the Foursquare API to explore the top venues of a given neighborhood.
3. CSV file "Geospatial_Coordinates.csv" given by IBM

# B. Clustering

### 1) Importing libraries

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

### 2) Importing table from wikipedia

In [5]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

data = pd.read_html(url, header = 0)

data = data[0]

### 3) Ignore cells with a borough that is Not assigned.

In [6]:
data = data[data['Borough'] != 'Not assigned']

In [7]:
data.columns = ['Postcode', 'Borough', 'Neighborhood']

### 4) If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [8]:
data[data['Neighborhood'] == 'Not assigned']

,Postcode,Borough,Neighborhood
9,M9A,Queen's Park,Not assigned


In [9]:
data.loc[9, 'Neighborhood'] = data.loc[9, 'Borough'] # Update the cell index 9
data.loc[9, 'Neighborhood']

"Queen's Park"

### 5) Combine rows with the same Postcode

In [10]:
data = (data.groupby(['Postcode', 'Borough'])['Neighborhood'].agg([('Neighborhood', ', '.join)]).reset_index())

In [11]:
data.shape

(103, 3)

### 6) Getting latitude and longitude for each row from file

In [12]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_3e43f34290ca4b54a33c57c3ed0cbcc3 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='pvNweBaBVE9tssBA63nW64WyqEJO-_X6aZxgQOb5pniJ',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_3e43f34290ca4b54a33c57c3ed0cbcc3.get_object(Bucket='pythonbasicsfordatascienceproject-donotdelete-pr-vsg1fzwopmijly',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

# If you are reading an Excel file into a pandas DataFrame, replace `read_csv` by `read_excel` in the next statement.
df_data_0 = pd.read_csv(body)
df_data_0.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [13]:
df_data_0.tail(1)

,Postal Code,Latitude,Longitude
102,M9W,43.706748,-79.594054


In [14]:
Latitude = df_data_0['Latitude']
Longitude = df_data_0['Longitude']

In [15]:
data['Latitude'] = Latitude
data['Longitude'] = Longitude

In [16]:
data.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## C. Explore the neighborhoods in Toronto

### 1. Import dependencies

In [17]:
!pip -q install folium
import folium 

In [18]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 

from geopy.geocoders import Nominatim 

import requests 


import matplotlib.cm as cm
import matplotlib.colors as colors

print('Libraries imported.')

Libraries imported.


In [19]:
address = "Toronto, On"

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [90]:
mapToronto = folium.Map(location = [latitude, longitude], zoom_start = 11)

for lat, long, borough, neighborhood in zip(data['Latitude'], data['Longitude'], data['Borough'], data['Neighborhood']):
    label = neighborhood + " " + borough
    label = folium.Popup(label, parse_html = True)
    
    folium.CircleMarker([lat, long], radius = 5, popup = label, color = 'green', 
                        fill = True, fill_color='#3186cc', fill_opacity=0.7, 
                        parse_html=False).add_to(mapToronto)
    
    
mapToronto

# D. Foursquare API

In [20]:
CLIENT_ID = 'CRZAKMKAWHVMJ25MK15JQI0QNAVXFG2240FR4KZKQUZ0MSMQ'
CLIENT_SECRET = 'WRVEG4QXC4OB0SEKQZI41NU4IBBXD1KWV2TGDXH5MP2OW04E' 
VERSION = '20180605' 

In [21]:
lat_test = data.loc[0, 'Latitude']
long_test = data.loc[0, 'Longitude']
name_test = data.loc[0, 'Borough']
print("Region: {}, latitude = {}, longitude = {}." .format(name_test, lat_test, long_test))

Region: Scarborough, latitude = 43.806686299999996, longitude = -79.19435340000001.


### 1) Check TOP 20 venues in the region (1km)

In [22]:
limit = 20
radius = 1000

# url with Venues/explore 
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID,
    CLIENT_SECRET,
    VERSION,
    lat_test,
    long_test,
    radius,
    limit)
print(url)

https://api.foursquare.com/v2/venues/explore?&client_id=CRZAKMKAWHVMJ25MK15JQI0QNAVXFG2240FR4KZKQUZ0MSMQ&client_secret=WRVEG4QXC4OB0SEKQZI41NU4IBBXD1KWV2TGDXH5MP2OW04E&v=20180605&ll=43.806686299999996,-79.19435340000001&radius=1000&limit=20


In [23]:
results = requests.get(url).json()

### 2) Functions to clean the results

In [24]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [25]:
venues = results['response']['groups'][0]['items']

from pandas.io.json import json_normalize
nearby_venues = json_normalize(venues) # flatten JSON

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Wendy's,Fast Food Restaurant,43.802008,-79.198080
1,Wendy's,Fast Food Restaurant,43.807448,-79.199056
2,Staples Morningside,Paper / Office Supplies Store,43.800285,-79.196607
3,Harvey's,Restaurant,43.800020,-79.198307
4,Caribbean Wave,Caribbean Restaurant,43.798558,-79.195777


### Getting the top 10 Venues for each neighborhood and creating a dataframe toronto_venues

In [26]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            

        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            

        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

toronto_venues = getNearbyVenues(names=data['Neighborhood'],
                                   latitudes=data['Latitude'],
                                   longitudes=data['Longitude']
                                  )

In [27]:
toronto_venues.shape

(1816, 7)

## E. Examin each neighborhood venues

In [44]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

In [45]:
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']

toronto_onehot = toronto_onehot.set_index('Neighborhood').reset_index()

toronto_onehot.head()

,Neighborhood,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Quad,College Rec Center,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Curling Ice,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundry Service,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Road,Rock Climbing Spot,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skating Rink,Ski Chalet,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stationery Store,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Track,Trail,Train Station,Transportation Service,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

## F. Filtering only columns of interest

In [46]:
toronto_onehot = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_onehot.head()

,Neighborhood,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Quad,College Rec Center,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Curling Ice,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundry Service,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Road,Rock Climbing Spot,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skating Rink,Ski Chalet,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stationery Store,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Track,Trail,Train Station,Transportation Service,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.0,0.0,0.0,0.05,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.05,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,

In [54]:
toronto_onehot[['Neighborhood','Breakfast Spot','Bubble Tea Shop','Café','Cantonese Restaurant','Chinese Restaurant','Coffee Shop','Tea Room']]

,Neighborhood,Breakfast Spot,Bubble Tea Shop,Café,Cantonese Restaurant,Chinese Restaurant,Coffee Shop,Tea Room
0,"Adelaide, King, Richmond",0.000000,0.00000,0.050000,0.00,0.000000,0.000000,0.00
1,Agincourt,0.050000,0.00000,0.000000,0.05,0.150000,0.000000,0.00
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.000000,0.00000,0.000000,0.00,0.150000,0.050000,0.00
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.000000,0.00000,0.000000,0.00,0.000000,0.052632,0.00
4,"Alderwood, Long Branch",0.000000,0.00000,0.000000,0.00,0.000000,0.050000,0.00
5,"Bathurst Manor, Downsview North, Wilson Heights",0.000000,0.00000,0.000000,0.00,0.000000,0.100000,0.00
6,Bayview Village,0.000000,0.00000,0.066667,0.00,0.066667,0.000000,0.00
7,"Bedford Park, Lawrence Manor East",0.000000,0.00000,0.050000,0.00,0.000000,0.150000,0.00
8,Berczy Park,0.000000,0.00000,0.000000,0.00,0.000000,0.000000,0.05
9,"Birch Cliff, Cliffside West",0.000000,0.00000,0.090909,0.00,0.000000,0.000000,0.00


In [55]:
toronto_tea = toronto_onehot[['Neighborhood', 'Breakfast Spot', 'Bubble Tea Shop', 'Café', 'Cantonese Restaurant', 'Chinese Restaurant', 'Coffee Shop', 'Tea Room']]

In [56]:
toronto_tea

,Neighborhood,Breakfast Spot,Bubble Tea Shop,Café,Cantonese Restaurant,Chinese Restaurant,Coffee Shop,Tea Room
0,"Adelaide, King, Richmond",0.000000,0.00000,0.050000,0.00,0.000000,0.000000,0.00
1,Agincourt,0.050000,0.00000,0.000000,0.05,0.150000,0.000000,0.00
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.000000,0.00000,0.000000,0.00,0.150000,0.050000,0.00
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.000000,0.00000,0.000000,0.00,0.000000,0.052632,0.00
4,"Alderwood, Long Branch",0.000000,0.00000,0.000000,0.00,0.000000,0.050000,0.00
5,"Bathurst Manor, Downsview North, Wilson Heights",0.000000,0.00000,0.000000,0.00,0.000000,0.100000,0.00
6,Bayview Village,0.000000,0.00000,0.066667,0.00,0.066667,0.000000,0.00
7,"Bedford Park, Lawrence Manor East",0.000000,0.00000,0.050000,0.00,0.000000,0.150000,0.00
8,Berczy Park,0.000000,0.00000,0.000000,0.00,0.000000,0.000000,0.05
9,"Birch Cliff, Cliffside West",0.000000,0.00000,0.090909,0.00,0.000000,0.000000,0.00


In [59]:
toronto_venues = toronto_onehot
toronto_venues['Total'] = toronto_venues.sum(axis = 1)

In [60]:
toronto_venues.sort_values(by=['Total'], ascending = False)

,Neighborhood,Breakfast Spot,Bubble Tea Shop,Café,Cantonese Restaurant,Chinese Restaurant,Coffee Shop,Tea Room,Total
27,"Commerce Court, Victoria Hotel",0.000000,0.00000,0.250000,0.00,0.000000,0.100000,0.05,0.400000
29,Davisville North,0.050000,0.00000,0.150000,0.00,0.000000,0.100000,0.05,0.350000
50,Harbourfront,0.100000,0.00000,0.000000,0.00,0.000000,0.250000,0.00,0.350000
44,"First Canadian Place, Underground city",0.000000,0.00000,0.200000,0.00,0.000000,0.150000,0.00,0.350000
35,"Dovercourt Village, Dufferin",0.000000,0.00000,0.200000,0.00,0.000000,0.150000,0.00,0.350000
32,"Design Exchange, Toronto Dominion Centre",0.000000,0.00000,0.150000,0.00,0.000000,0.200000,0.00,0.350000
97,Woburn,0.000000,0.00000,0.000000,0.00,0.111111,0.222222,0.00,0.333333
14,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.000000,0.00000,0.125000,0.00,0.000000,0.187500,0.00,0.312500
63,Lawrence Park,0.000000,0.00000,0.200000,0.00,0.000000,0.100000,0.00,0.300000
61,L'Amoreaux West,0.050000,0.05000,0.000000,0.00,0.100000,0.100000,0.00,0.300000


In [62]:
toronto_venues.shape

(101, 9)

In [63]:
data.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [64]:
new_df = pd.merge(data, toronto_venues, on = 'Neighborhood')

In [65]:
new_df.sort_values(by = ['Total'], ascending = False)

,Postcode,Borough,Neighborhood,Latitude,Longitude,Breakfast Spot,Bubble Tea Shop,Café,Cantonese Restaurant,Chinese Restaurant,Coffee Shop,Tea Room,Total
60,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817,0.000000,0.00000,0.250000,0.00,0.000000,0.100000,0.05,0.400000
44,M4P,Central Toronto,Davisville North,43.712751,-79.390197,0.050000,0.00000,0.150000,0.00,0.000000,0.100000,0.05,0.350000
59,M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",43.647177,-79.381576,0.000000,0.00000,0.150000,0.00,0.000000,0.200000,0.00,0.350000
52,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,0.100000,0.00000,0.000000,0.00,0.000000,0.250000,0.00,0.350000
75,M6H,West Toronto,"Dovercourt Village, Dufferin",43.669005,-79.442259,0.000000,0.00000,0.200000,0.00,0.000000,0.150000,0.00,0.350000
69,M5X,Downtown Toronto,"First Canadian Place, Underground city",43.648429,-79.382280,0.000000,0.00000,0.200000,0.00,0.000000,0.150000,0.00,0.350000
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0.000000,0.00000,0.000000,0.00,0.111111,0.222222,0.00,0.333333
67,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo...",43.628947,-79.394420,0.000000,0.00000,0.125000,0.00,0.000000,0.187500,0.00,0.312500
43,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0.000000,0.00000,0.200000,0.00,0.000000,0.100000,0.00,0.300000
15,M1W,Scarborough,L'Amoreaux West,43.799525,-79.318389,0.050000,0.05000,0.000000,0.00,0.100000,0.100000,0.00,0.300000


In [66]:
mapToronto = folium.Map(location = [43.653963, -79.387207], zoom_start = 11)
for neighborhood, lat, long, total in zip(new_df['Neighborhood'], new_df['Latitude'], new_df['Longitude'], new_df['Total']):
    label = neighborhood + ' score: ' + str(round(total, 3))
    label = folium.Popup(label, parse_html = True)
    
    folium.CircleMarker([lat, long], radius = 5, popup = label, color = 'red', fill=True, fill_opacity=1).add_to(mapToronto)

mapToronto

## Now that we have the score of popularity for each neighborhood. We can now set the area where popularity of this type of shop is greater than 0.2

In [67]:
restricted_df = new_df[new_df['Total'] > 0.2]
restricted_df.sort_values(by = 'Total', ascending = False)

,Postcode,Borough,Neighborhood,Latitude,Longitude,Breakfast Spot,Bubble Tea Shop,Café,Cantonese Restaurant,Chinese Restaurant,Coffee Shop,Tea Room,Total
60,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817,0.00,0.00000,0.250000,0.00,0.000000,0.100000,0.05,0.400000
44,M4P,Central Toronto,Davisville North,43.712751,-79.390197,0.05,0.00000,0.150000,0.00,0.000000,0.100000,0.05,0.350000
52,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,0.10,0.00000,0.000000,0.00,0.000000,0.250000,0.00,0.350000
75,M6H,West Toronto,"Dovercourt Village, Dufferin",43.669005,-79.442259,0.00,0.00000,0.200000,0.00,0.000000,0.150000,0.00,0.350000
69,M5X,Downtown Toronto,"First Canadian Place, Underground city",43.648429,-79.382280,0.00,0.00000,0.200000,0.00,0.000000,0.150000,0.00,0.350000
59,M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",43.647177,-79.381576,0.00,0.00000,0.150000,0.00,0.000000,0.200000,0.00,0.350000
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0.00,0.00000,0.000000,0.00,0.111111,0.222222,0.00,0.333333
67,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo...",43.628947,-79.394420,0.00,0.00000,0.125000,0.00,0.000000,0.187500,0.00,0.312500
15,M1W,Scarborough,L'Amoreaux West,43.799525,-79.318389,0.05,0.05000,0.000000,0.00,0.100000,0.100000,0.00,0.300000
43,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0.00,0.00000,0.200000,0.00,0.000000,0.100000,0.00,0.300000


In [68]:
restricted_df.shape

(25, 13)

In [69]:
restrictedMap = folium.Map(location = [43.653963, -79.387207], zoom_start = 11)

for lat, long in zip(restricted_df['Latitude'], restricted_df['Longitude']):
     folium.CircleMarker([lat, long], radius = 5, color = 'red', 
                         fill=True, fill_opacity=1).add_to(restrictedMap)
restrictedMap

In [81]:
cluster_df = new_df[['Total']]

In [82]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters = 3, init = 'k-means++')

In [83]:
labels = kmeans.fit_predict(cluster_df)

In [84]:
new_df['Cluster'] = labels

In [85]:
new_df.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude,Breakfast Spot,Bubble Tea Shop,Café,Cantonese Restaurant,Chinese Restaurant,Coffee Shop,Tea Room,Total,Cluster
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,0.0,0.0,0.0,0.0,0.062500,0.062500,0.0,0.125000,0
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,0.2,0.0,0.0,0.0,0.000000,0.000000,0.0,0.200000,0
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0.0,0.0,0.0,0.0,0.000000,0.050000,0.0,0.050000,1
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0.0,0.0,0.0,0.0,0.111111,0.222222,0.0,0.333333,2
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0.0,0.0,0.0,0.0,0.050000,0.150000,0.0,0.200000,0


In [86]:
cluster_0 = new_df[new_df['Cluster'] == 0]

clusteredMap = folium.Map(location = [43.653963, -79.387207], zoom_start = 11)
for lat, long in zip(cluster_0['Latitude'], cluster_0['Longitude']):
    folium.CircleMarker([lat, long], radius = 5, color = 'blue', fill = True, fill_opacity = 1).add_to(clusteredMap)

In [87]:
cluster_1 = new_df[new_df['Cluster'] == 1]
for lat, long in zip(cluster_1['Latitude'], cluster_1['Longitude']):
    folium.CircleMarker([lat, long], radius = 5, color = 'green', fill = True, fill_opacity = 1).add_to(clusteredMap)

In [88]:
cluster_2 = new_df[new_df['Cluster'] == 2]
for lat, long in zip(cluster_2['Latitude'], cluster_2['Longitude']):
    folium.CircleMarker([lat, long], radius = 5, color = 'red', fill = True, fill_opacity = 1).add_to(clusteredMap)

In [89]:
clusteredMap